# TensorRT YoloV5s Sample Application


--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Import model](#Import-model)
1. [Write and test app code](#Write-and-test-app-code-in-notebook)
1. [Package app](#Package-app)
1. [Deploy app to device](#Deploy-app-to-device)

# Prerequisites (DO NOT SKIP)

1. **PLEASE READ THE [README](README.md) INCLUDE WITH THIS BEFORE YOU START USING THIS NOTEBOOK**

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama', region_name = 'us-west-2')

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# application name
app_name = 'lab3'

## package names and node names
code_package_name = 'lab3'
camera_node_name = 'abstract_rtsp_media_source'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./lab3 && panorama-cli import-application

## Download Depedencies and Artifacts (One Time Download)

In [ ]:
panorama_test_utility.download_artifacts_gpu_sample('lab3', account_id)

## Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

We used an ```abstract data source``` here, usually this lets you select the pre-created camera source from the console. But programatically, we have to do the following steps


- Create Camera
- Create Override json file
- Include the Override json file while are deploying the application

### Create New Camera

Because we are using an ```abstract_rtsp_media_source```, we have to create a camera before we can use the ```abstract_rtsp_media_source```

**NOTE** : Update your RTSP Info in the next cell, Username, Password and RTSP Stream URL

In [ ]:
CAMERA_NAME = "test_rtsp_camera_lab3"

CAMERA_CREDS = '{"Username":"root","Password":"Aws2017!","StreamUrl": "rtsp://10.92.202.65/onvif-media/media.amp?profile=profile_1_h264&sessiontimeout=60&streamtype=unicast"}'

In [ ]:
res = !aws panorama create-node-from-template-job --template-type RTSP_CAMERA_STREAM \
    --output-package-name {CAMERA_NAME} \
    --output-package-version '1.0' \
    --node-name {CAMERA_NAME} \
    --template-parameters '{CAMERA_CREDS}'

# FIXME : camera node creation fails if it already exists.
# Should either ignore the already-exist error, or delete the node at the end of this notebook

res = ''.join(res)
print(res)
res_json = json.loads(res)

In [ ]:
!aws panorama describe-node-from-template-job --job-id {res_json['JobId']}

### Upload application to Panorama for deploying to devices

In [ ]:
container_asset_name = 'lab3'

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./lab3 && pwd && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

### How to deploy to the device?

The [README](README.md) file has detailed instructions on how to deploy. Please go to the deployment section of the README file and follow along the instructions